In [13]:
import pandas as pd
import numpy as np
import time

global no_of_transactions
global ms
global m

data =pd.read_csv("docword."+"kos"+".txt",
                  skiprows=3,
                  header=None,
                  sep= " ",
                  names=["docId","wordId","count"])

data_clean=data.groupby(["docId","wordId"])["count"].sum().unstack().reset_index().fillna(0).set_index("docId")

df1=data_clean.copy()
data_clean[data_clean>0]=1
word_items=data_clean.columns


no_of_transactions=max(data_clean.index)

#set of all transactions

Transactions=[]
for id in data_clean.index:
    ls=np.flatnonzero(data_clean.loc[id]).tolist()
    ls1=[item+1 for item in ls]
    
    ls2=set(ls1)
    Transactions.append(ls2)
#calculating mis of every word
def mis_individual():
    
    mis=[]
    
    
    for item in range(1,max(word_items)+1):
        if (item in word_items):
            mis.append(max((df1[item].sum())/no_of_transactions,0.05))      ## mis values less than 0.05 are ignored             
        else :                                                              # Note that here df1 is used 
                                                                            # see also defination ms_apriori 
                    mis.append(0.05)                                        # if condition for appending item to f
                                                                            # where data_clean is used instead
    return mis
#sortin and storing word_incices into m accotrding to mis
def sort_accord_mis(data):
    
    vals=np.array(data)
    vals1=np.argsort(vals).tolist()
    vals2=[item+1 for item in vals1]
    
    return vals2

ms= mis_individual()                           #contains all the mis
m=sort_accord_mis(ms)                          #contains all the sorted 1-item sets according to mis


## Not required as elements of f are already sorted in desired manner ##
def level_2_candidate_gen(m,phi):
    
    c_set=[]
    for i in range(0,len(m)):
        
        if ((data_clean[m[i]].sum())/no_of_transactions)>=ms[m[i]-1]:
           
            for j in range(i+1,len(m)) :
               
                
                if (((data_clean[m[j]].sum())/no_of_transactions)>=ms[m[i]-1]):
                        if(abs(((data_clean[m[i]].sum())/no_of_transactions)-((data_clean[m[j]].sum())/no_of_transactions))<=phi):
                                c_set.append({m[i],m[j]})
                        
                        
    return  c_set  

#candidate-gen function for multiple minimum support

def MS_candidate_gen(f,phi):
        
        c_set=[]
        c=set()
        for f1 in f:
            for f2 in f :
                
                max_f1=max(f1)
                max_f2=max(f2)
                if ((f2.difference(f1)).union(f1.difference(f2))=={max_f1,max_f2}):
                    if(max_f1<max_f2):
                        if(abs(((data_clean[max_f1].sum())/no_of_transactions)-((data_clean[max_f2].sum())/no_of_transactions))<=phi):

                            c =f1.union({max_f2})
                            c_set.append(c)
                    
                    for i in c:
                        if c.difference({i}) not in f:
                            
                            temp=c.difference({i})
                            if (min(c) in temp):
                            
                                if (c_set != []) & (c in c_set):
                               
                                      c_set.remove(c)
                            elif(ms[min(c)-1]==ms[min(c.difference({min(c)}))]):
                                if (c_set != []) & (c in c_set):
                               
                                      c_set.remove(c)                             
        return c_set

#counts the frequency of a candidate set

def support_count_of_sets(input_set):
    count=0
    for t in Transactions:
        if input_set.issubset(t):
            count =count+1
    return count


#converts the frequent itemset list to words vocubaliry file

def to_word_conversion(word_index):
    
    vocabulary_file=open("vocab."+"kos"+".txt","r")
    vocabulary=vocabulary_file.read().split("\n")
    frequent_set_words=[]
    for frequent_set in word_index:
        temp_word_set=set()
        for index in frequent_set:
            temp_word_set.add(vocabulary[index-1])
        frequent_set_words.append(temp_word_set)
    return frequent_set_words
            
def MS_apriori(data_clean,k,phi):
    
    # defining global variabe as to be used in output_file function
    
    global K
    global Phi
    global total_time
    global final_word_list
    global final_f
    
    f=[]
    start_time=time.time()
    
    for item in m:
        if item in word_items:
            
            if ((data_clean[item].sum())/no_of_transactions)>=ms[item-1]:
                f.append(set([item]))
    
    final_f=[]
    final_f=final_f+f
    print(len(f))
   
    
    for i in range(2,k+1):
        
        c_set=MS_candidate_gen(f,phi)
        temp=len(f)
       
        for item in c_set:
            
            if (support_count_of_sets(item) /no_of_transactions)>=ms[min(item)-1]:
                f.append(item)                           #  now f contains k-1,k length frequent item sets
                             
        f=f[temp:]                                       # this removes k-1 length frequent item sets from f
        final_f=final_f+f                                # appends all k length frequent items to f_final

    end_time  =time.time() 
    total_time=end_time-start_time
    K=k
    Phi=phi
    final_word_list=to_word_conversion(final_f)
    
    return final_f

def output_file_MS_apriori(k,phi):
    MS_apriori(data_clean,k,phi)
    output_file=open("output_MS_Apriori_kos.txt","a+")
    output=""
    output=output+"\n\n"+"K="+str(K)+"\n\n"+"phi="+str(Phi)+"\n\n"+"Runtime is :"+str(total_time)+" seconds"+"\n\n"
    output=output+"The Frequent sets are:"+"\n\n"+"Word indices:"+"\n\n"
    output_file.write(output)
    output_file.write(str(final_f))
    output_file.write("\n\n"+"words:"+"\n\n")
    output_file.write(str(final_word_list))
    print("K=",k)
    print("Runtime is :"+str(total_time)+" seconds")
    print("The Frequent sets are:")
    print(final_word_list)
#############################################################################
output_file_MS_apriori(2,0.001)
output_file_MS_apriori(3,0.001)
output_file_MS_apriori(4,0.001)
output_file_MS_apriori(5,0.001)

24
K= 2
Runtime is :1.9596245288848877 seconds
The Frequent sets are:
[{'mar'}, {'materiel'}, {'hstewart'}, {'apr'}, {'endspan'}, {'dryfly'}, {'republicansforkerry'}, {'openhttpwwwedwardsforprezcomdailykoshtml'}, {'racine'}, {'var'}, {'newwindow'}, {'startspan'}, {'calistan'}, {'calculator'}, {'bushsux'}, {'altsite'}, {'boxrdf_feeds'}, {'boxfeed_listing'}, {'montclair'}, {'boxblogroll'}, {'admin'}, {'asap'}, {'function'}, {'sep'}, {'materiel', 'hstewart'}, {'republicansforkerry', 'endspan'}, {'openhttpwwwedwardsforprezcomdailykoshtml', 'endspan'}, {'racine', 'endspan'}, {'var', 'endspan'}, {'newwindow', 'endspan'}, {'startspan', 'endspan'}, {'montclair', 'endspan'}, {'dryfly', 'endspan'}, {'republicansforkerry', 'dryfly'}, {'openhttpwwwedwardsforprezcomdailykoshtml', 'dryfly'}, {'racine', 'dryfly'}, {'var', 'dryfly'}, {'newwindow', 'dryfly'}, {'dryfly', 'startspan'}, {'dryfly', 'montclair'}, {'republicansforkerry', 'var'}, {'republicansforkerry', 'startspan'}, {'republicansforkerry', '

24
K= 4
Runtime is :5.686636686325073 seconds
The Frequent sets are:
[{'mar'}, {'materiel'}, {'hstewart'}, {'apr'}, {'endspan'}, {'dryfly'}, {'republicansforkerry'}, {'openhttpwwwedwardsforprezcomdailykoshtml'}, {'racine'}, {'var'}, {'newwindow'}, {'startspan'}, {'calistan'}, {'calculator'}, {'bushsux'}, {'altsite'}, {'boxrdf_feeds'}, {'boxfeed_listing'}, {'montclair'}, {'boxblogroll'}, {'admin'}, {'asap'}, {'function'}, {'sep'}, {'materiel', 'hstewart'}, {'republicansforkerry', 'endspan'}, {'openhttpwwwedwardsforprezcomdailykoshtml', 'endspan'}, {'racine', 'endspan'}, {'var', 'endspan'}, {'newwindow', 'endspan'}, {'startspan', 'endspan'}, {'montclair', 'endspan'}, {'dryfly', 'endspan'}, {'republicansforkerry', 'dryfly'}, {'openhttpwwwedwardsforprezcomdailykoshtml', 'dryfly'}, {'racine', 'dryfly'}, {'var', 'dryfly'}, {'newwindow', 'dryfly'}, {'dryfly', 'startspan'}, {'dryfly', 'montclair'}, {'republicansforkerry', 'var'}, {'republicansforkerry', 'startspan'}, {'republicansforkerry', 'o

24
K= 5
Runtime is :22.83659029006958 seconds
The Frequent sets are:
[{'mar'}, {'materiel'}, {'hstewart'}, {'apr'}, {'endspan'}, {'dryfly'}, {'republicansforkerry'}, {'openhttpwwwedwardsforprezcomdailykoshtml'}, {'racine'}, {'var'}, {'newwindow'}, {'startspan'}, {'calistan'}, {'calculator'}, {'bushsux'}, {'altsite'}, {'boxrdf_feeds'}, {'boxfeed_listing'}, {'montclair'}, {'boxblogroll'}, {'admin'}, {'asap'}, {'function'}, {'sep'}, {'materiel', 'hstewart'}, {'republicansforkerry', 'endspan'}, {'openhttpwwwedwardsforprezcomdailykoshtml', 'endspan'}, {'racine', 'endspan'}, {'var', 'endspan'}, {'newwindow', 'endspan'}, {'startspan', 'endspan'}, {'montclair', 'endspan'}, {'dryfly', 'endspan'}, {'republicansforkerry', 'dryfly'}, {'openhttpwwwedwardsforprezcomdailykoshtml', 'dryfly'}, {'racine', 'dryfly'}, {'var', 'dryfly'}, {'newwindow', 'dryfly'}, {'dryfly', 'startspan'}, {'dryfly', 'montclair'}, {'republicansforkerry', 'var'}, {'republicansforkerry', 'startspan'}, {'republicansforkerry', 'o

In [2]:
f=[]
mis_alter1=[]
mis_alter2=[]
for item in m:
    if ((data_clean[item].sum())/no_of_transactions)>=ms[item-1]:
        f.append(item)


In [3]:
for i in f:
    #print(i)
    mis_alter1.append((df1[i].sum())/no_of_transactions)
for i in f: 
    mis_alter2.append((data_clean[i].sum())/no_of_transactions)
print(mis_alter2)


[0.05860058309037901, 0.07492711370262391, 0.07492711370262391, 0.07725947521865889, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09737609329446065, 0.09737609329446065, 0.0988338192419825, 0.10583090379008746]


In [4]:
print(mis_alter1)

[0.05860058309037901, 0.07492711370262391, 0.07492711370262391, 0.07725947521865889, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09650145772594752, 0.09737609329446065, 0.09737609329446065, 0.0988338192419825, 0.10583090379008746]


In [5]:
mis_diff=[]
print(len(f))
print(len(mis_alter1))
for i in range(0,len(f)):
    #print(mis_alter2[i]-mis_alter1[i])
    mis_diff.append(mis_alter2[i]-mis_alter1[i])
print(mis_diff)

len(mis_diff)

24
24
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


24

In [55]:
print(mis_alter2[2075])
mis_alter2[1273]-mis_alter1[1273]

0.004664723032069971


0.5131195335276968

In [96]:
type(min({2,3,5}))

int

In [94]:
type(5)
s={2,3,4,5,7}

int

In [108]:
if min(9,3) in {2,3,5}:
    print("r")

r
